<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [3]:
import folium
import wget
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [166]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [13]:
#original file download
# filepath ='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv'
# df_original = pd.read_csv(filepath, header=0)
file_path = r"C:\Users\Raegan\Downloads\AppliedDataScinece_SpaceXProj_FoliumGeo_Part6.csv"  #remname file
try:
    spacex_df.to_csv(file_path, index=False)
    print(f'DataFrame saved to {file_path}')
except Exception as e:
    print(f'An error occurred: {e}')

DataFrame saved to C:\Users\Raegan\Downloads\AppliedDataScinece_SpaceXProj_FoliumGeo_Part6.csv


In [168]:
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [170]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [172]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [174]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#4169E1', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name - click on marker to see
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#4169E1;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [176]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#4169E1',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a marker for NASA Johnson Space Center
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#4169E1;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Add the NASA Johnson Space Center Circle and Marker to the map
site_map.add_child(circle)
site_map.add_child(marker)

# site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Loop through the SpaceX DataFrame and add Circles for each launch site
for index, row in launch_sites_df.iterrows():
    # Create a Circle for each launch site
    site_circle = folium.Circle(
        location=(row['Lat'], row['Long']),
        radius=1000,  # Radius in meters
        color='#4169E1',  # Blue color
        fill=True,
        fill_opacity=0.5
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Create a Marker for each launch site
    site_marker = folium.Marker(
        location=(row['Lat'], row['Long']),
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#4169E1;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    
    # Add the Circle and Marker to the map
    site_map.add_child(site_circle)
    site_map.add_child(site_marker)

# Save the map to an HTML file or display it
site_map.save("spacex_launch_sites_map.html")
site_map  # This line will display the map in a Jupyter Notebook



The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


## Increased launch efficiency- launch sites closer to the equator benefit from Earths rotational speed - provides additional boost reaching orbit more efficeintly and with less fuel (cost savings and heavier payloads)
## Access to desired orbits- easiler access to a variety of orbital inclinations (reduces extensive maneuvering after laundh)
## Saftey and Risk Mitigation- Costal proximity minimizes risk to populated areas in event of launch failure and adds possiblility of recovery operations for reusable rocket components- especially when drones are stationed in ocean)
## Operational Flexibility- equator can have a wider range of launch windows and weather conditions are more favorable leading to higher success launches- and reliable launch schedules
## Geopolitical considerations and implications- international agreements, partnerships, security ocnsiderations
## enabling the company to optimize its launch operations and support a wide range of satellite deployment and space exploration objectives


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [178]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [180]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [190]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [194]:
# Initialize the map
# nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch result
for index, row in spacex_df.iterrows():
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color']),
        popup=f"Launch Site: {row['Launch Site']}<br>Success: {'Yes' if row['class'] == 1 else 'No'}"
    )
    marker_cluster.add_child(marker)

# Display the map
site_map

In [200]:
spacex_df.head(10)

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


In [ ]:
# # to see the success and failures zoom into location on map and click
# # Initialize the map and marker cluster
# # nasa_coordinate = [29.559684888503615, -95.0830971930759]
# site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# # marker_cluster = MarkerCluster().add_to(site_map)

# # Add markers for each launch result in spacex_df
# for index, record in spacex_df.iterrows():
#     # Get the coordinates
#     coordinate = (record['Lat'], record['Long'])
    
#     # Retrieve the marker color directly from the DataFrame
#     marker_color = record['marker_color']
    
#     # Create and customize the Marker
#     marker = folium.Marker(
#         location=coordinate,
#         icon=folium.Icon(color='white', icon_color=marker_color),
#         popup=f"{record['Launch Site']}: {'Success' if record['class'] == 1 else 'Failure'}"
#     )
    
#     # Add the Marker to the marker cluster
#     marker_cluster.add_child(marker)

# # Add the marker cluster to the site map
# site_map.add_child(marker_cluster)

# # Display the map
# site_map.save("spacex_launchsuccee_failure_map.html")  # Uncomment this line if you want to save the map
# site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [103]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map- on top right corner of map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [113]:
import math
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [149]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Sample coastline coordinates (for demonstration purposes)
# coastlines = {
#     'Coastline': ['Coastline 1', 'Coastline 2', 'Coastline 3'],
#     'Lat': [28.56367, 28.558, 34.740],
#     'Lon': [-80.57163, -80.560, -120.600]
# }
coastlines = {
    'Coastline': ['Coastline 1'],
    'Lat': [28.56311],
    'Lon': [-80.56797]
}
coastline_df = pd.DataFrame(coastlines)

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Find closest coastline for each launch site
for index, launch_site in launch_sites_df.iterrows():
    launch_lat = launch_site['Lat']
    launch_lon = launch_site['Long']
    
    # Initialize minimum distance and closest coastline
    min_distance = float('inf')
    closest_coastline = None
    
    # Check distance to each coastline
    for _, coastline in coastline_df.iterrows():
        coastline_lat = coastline['Lat']
        coastline_lon = coastline['Lon']
        
        distance = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)
        
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coastline['Coastline']
    
    print(f"The distance of Launch Site: {launch_site['Launch Site']} to CoastLine: {closest_coastline} is {min_distance:.2f} km.")

The distance of Launch Site: CCAFS LC-40 to CoastLine: Coastline 1 is 0.92 km.
The distance of Launch Site: CCAFS SLC-40 to CoastLine: Coastline 1 is 0.86 km.
The distance of Launch Site: KSC LC-39A to CoastLine: Coastline 1 is 7.79 km.
The distance of Launch Site: VAFB SLC-4E to CoastLine: Coastline 1 is 3826.67 km.


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [151]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Find closest coastline for a specific launch site (e.g., Cape Canaveral)
selected_launch_site = launch_sites_df.iloc[0]  # Select Cape Canaveral
launch_lat = selected_launch_site['Lat']
launch_lon = selected_launch_site['Long']

# Initialize minimum distance and closest coastline
min_distance = float('inf')
closest_coastline = None
closest_coords = None

# Check distance to each coastline
for _, coastline in coastline_df.iterrows():
    coastline_lat = coastline['Lat']
    coastline_lon = coastline['Lon']
    
    distance = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)
    
    if distance < min_distance:
        min_distance = distance
        closest_coastline = coastline['Coastline']
        closest_coords = (coastline_lat, coastline_lon)

# Create a Marker for the closest coastline
if closest_coords:
    distance_marker = folium.Marker(
        closest_coords,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(min_distance),
        )
    )

    # Add the distance marker to the site map
    site_map.add_child(distance_marker)

# Optional: Add a marker for the selected launch site for reference
folium.Marker(
    [launch_lat, launch_lon],
    popup=selected_launch_site['Launch Site'],
    icon=folium.Icon(color='blue')
).add_to(site_map)

 
# Zoom into the distance marker's coordinates
site_map.location = closest_coords  # Set map center to the distance marker
site_map.zoom_start = 20  # Adjust the zoom level (higher values for closer zoom)

# Display the map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [153]:
# Draw a PolyLine from the launch site to the closest coastline point
lines= folium.PolyLine(
        locations=[(launch_lat, launch_lon), closest_coords],
        color='blue',
        weight=2,
        opacity=0.8
)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [155]:
# import folium
# from folium.map import DivIcon
# import pandas as pd
# import math

# Sample DataFrame with launch site coordinates
# launch_sites = {
#     'Launch Site': ['Cape Canaveral', 'Kennedy Space Center', 'Vandenberg Space Force Base', 'SpaceX Boca Chica'],
#     'Lat': [28.5623, 28.5721, 34.7364, 25.9972],
#     'Long': [-80.5774, -80.6480, -120.5734, -97.1560]
# }
# launch_sites_df = pd.DataFrame(launch_sites)

# Sample DataFrame for additional points (cities, railways, highways)
additional_points = {
    'Type': ['City', 'Railway', 'Highway'],
    'Name': ['Melbourne', 'Random Railway', 'Random Hwy'],
    'Lat': [28.07872, 28.57017, 28.57227],
    'Long': [-80.60505, -80.80368, -80.79825]
}
points_df = pd.DataFrame(additional_points)

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Initialize the map centered around a general location
# nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Find the closest additional point for a specific launch site (e.g., Cape Canaveral)
selected_launch_site = launch_sites_df.iloc[0]  # Select Cape Canaveral
launch_lat = selected_launch_site['Lat']
launch_lon = selected_launch_site['Long']

# Initialize minimum distance and closest point
min_distance = float('inf')
closest_point = None
closest_coords = None

# Check distance to each additional point
for _, point in points_df.iterrows():
    point_lat = point['Lat']
    point_lon = point['Long']
    
    distance = calculate_distance(launch_lat, launch_lon, point_lat, point_lon)
    
    if distance < min_distance:
        min_distance = distance
        closest_point = point['Name']
        closest_coords = (point_lat, point_lon)

# Create a Marker for the closest additional point
if closest_coords:
    # Create a distance marker
    distance_marker = folium.Marker(
        closest_coords,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(min_distance),
        )
    )

    # Add the distance marker to the site map
    site_map.add_child(distance_marker)

    # Draw a PolyLine from the launch site to the closest additional point
    folium.PolyLine(
        locations=[(launch_lat, launch_lon), closest_coords],
        color='blue',
        weight=2,
        opacity=0.8
    ).add_to(site_map)

# Optional: Add a marker for the selected launch site for reference
folium.Marker(
    [launch_lat, launch_lon],
    popup=selected_launch_site['Launch Site'],
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
